In [1]:
# Chinese Chess (Xiangqi) with AlphaZero

# This notebook demonstrates how to use the AlphaZero model to play Chinese Chess.

In [4]:
# Import necessary modules
import numpy as np
import torch

In [5]:
# Import our custom modules
from aznet import load_model, AZNet
from board import init_board, print_board, board_to_state, state_to_board, make_move
from moves import move_to_uci, uci_to_move, get_all_moves
from alphazero import AlphaZero
from utils import play_game, play_interactive

In [16]:
# Because of the limited computational power we have trained our model with A100's in Google Collab
# The main_notebook file consists of training results and self-play game resuls
# This notebook is polished version of the notebook that has been used for training

# Load the AlphaZero Model
#### First, let's load the pre-trained model

In [10]:
# Load the model
print("Loading AlphaZero model...")
model = load_model("aznet_chinese_chess (1).pth")

Loading AlphaZero model...


C:\Users\berky\rl_project_new\aznet.py:79: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


In [12]:
# Create the AlphaZero agent
alpha_zero = AlphaZero(model, num_simulations=100)

# Test Board Representation
### Let's test the board representation functions.

In [25]:
# Initialize the board
board = init_board()

# Print the board
print_board(board)

# Convert to state and back
state = board_to_state(board)
board_reconstructed = state_to_board(state)

# Verify reconstruction is correct
assert board == board_reconstructed, "Board reconstruction failed"
print("Board representation test passed!")


    a   b   c   d   e   f   g   h   i
  ┌───┬───┬───┬───┬───┬───┬───┬───┬───┐
10 │ r │ n │ b │ a │ k │ a │ b │ n │ r │
  ├───┼───┼───┼───┼───┼───┼───┼───┼───┤
9 │ . │ . │ . │ . │ . │ . │ . │ . │ . │
  ├───┼───┼───┼───┼───┼───┼───┼───┼───┤
8 │ . │ c │ . │ . │ . │ c │ . │ . │ . │
  ├───┼───┼───┼───┼───┼───┼───┼───┼───┤
7 │ p │ . │ p │ . │ p │ . │ p │ . │ p │
  ├───┼───┼───┼───┼───┼───┼───┼───┼───┤
6 │ . │ . │ . │ . │ . │ . │ . │ . │ . │
  ├───┼───┼───┼───┼───┼───┼───┼───┼───┤
5 │ . │ . │ . │ . │ . │ . │ . │ . │ . │
  ├───┼───┼───┼───┼───┼───┼───┼───┼───┤
4 │ P │ . │ P │ . │ P │ . │ P │ . │ P │
  ├───┼───┼───┼───┼───┼───┼───┼───┼───┤
3 │ . │ C │ . │ . │ . │ C │ . │ . │ . │
  ├───┼───┼───┼───┼───┼───┼───┼───┼───┤
2 │ . │ . │ . │ . │ . │ . │ . │ . │ . │
  ├───┼───┼───┼───┼───┼───┼───┼───┼───┤
1 │ R │ N │ B │ A │ K │ A │ B │ N │ R │
  └───┴───┴───┴───┴───┴───┴───┴───┴───┘
Board representation test passed!


# Play Options

### Now we can choose to watch the AI play against itself or play against it

In [28]:
def main():
    choice = input("""
Choose an option:
1. Watch AlphaZero play against itself
2. Play against AlphaZero (you as Black)
Enter your choice (1-2): """)
    
    if choice == '1':
        num_moves = int(input("Enter number of moves to play (default 20): ") or 20)
        play_game(model, alpha_zero, num_moves)
    elif choice == '2':
        play_interactive(model, alpha_zero)
    else:
        print("Invalid choice.")

# Examine the Neural Network
### Let's examine the structure of the neural network

In [31]:
# Print model summary
print("Neural Network Architecture:")
print(model)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params:,}")

Neural Network Architecture:
AZNet(
  (conv1): Conv2d(15, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (res_blocks): Sequential(
    (0): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), 

# Test a Single Move
### Let's test the AlphaZero agent by getting a single move suggestion.

In [14]:
# Initialize a new board
import importlib
import mcts
importlib.reload(mcts)
from mcts import MCTSNode

board = init_board()
state = board_to_state(board)

# Get a move from AlphaZero
print("AlphaZero is thinking...")
move = alpha_zero.select_move(state)
uci = move_to_uci(move)
print(f"AlphaZero suggests: {uci}")

AlphaZero is thinking...
AlphaZero suggests: c4-c5


In [18]:
# As you can see here our pre-trained model suggests the moves c4-c5

In [20]:
# The training is also done in Google Collab with the provided functions